In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("spotify_millsongdata.csv")

In [ ]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [ ]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [ ]:
df.shape

(57650, 4)

In [ ]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [ ]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [ ]:
df.head(10)

,artist,song,text
0,Olivia Newton-John,Away In A Manger,"Away in a manger, no cry for a bed \r\nThe li..."
1,Nirvana,"Yes, She's My Skinhead",I said I want to meet you aah \r\nI said I wa...
2,Pretenders,Mystery Achievement,Mystery achievement \r\nDon't breathe down my...
3,Christmas Songs,"Fum, Fum, Fum","On this joyful Christmas Day \r\nSing fum, fu..."
4,Phish,Sleep,I can't describe the feeling when \r\n \r\nI...
5,Indiana Bible College,Better Is One Day Medley,[Chorus:] \r\nBetter is one day in Your court...
6,Frank Zappa,Heavenly Bank Account,"Frank zappa (lead guitar, vocals) \r\nIke wil..."
7,Christmas Songs,"O Come, O Come, Emmanuel","O come, O come, Emmanuel \r\nAnd ransom capti..."
8,Nicki Minaj,Whip My Hair Back And Fourth,"I was on my way to school \r\nI do'z me, \r\..."
9,Faith No More,Ricochet,All of that thick time without you \r\nHas ma...


In [ ]:
df['text'][0]

'Away in a manger, no cry for a bed  \r\nThe little load jesus laid down his sweet head  \r\nThe stars in the bright sky, look down where he lay  \r\nThe little load jesus, asleep on the hay  \r\n  \r\nThe cattle are lowing, the baby awakes  \r\nBut little load jesaus, no crying he makes  \r\nI love thee load jesus, look down from the sky  \r\nAnd stay by my side until morning is nigh  \r\n  \r\nBe near me, load jesus, I ask thee to stay  \r\nClose by me forever, and love me I pray  \r\nBless all the dear children in thy tender care  \r\nAnd take us to heaven, to live with thee there  \r\n  \r\nAnd takes us to heaven, to live with thee there\r\n\r\n'

In [ ]:
df = df.sample(5000)

In [ ]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [ ]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [ ]:
similarity[0]

array([1.        , 0.01178699, 0.02507467, ..., 0.063002  , 0.03000229,
       0.0639102 ])

In [ ]:
df[df['song'] == 'Crying Over You']

,artist,song,text


from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_0.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

In [ ]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)

    return songs

In [1]:
recommendation('Crying Over You')

In [ ]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))